In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch import nn
import numpy as np
output_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_wav2vec.csv'

# Load your features and labels from your CSV
features_df = pd.read_csv(output_path, index_col=0)
X = features_df.drop('genre', axis=1).values
y = pd.factorize(features_df['genre'])[0]  # Convert genre labels to integer indices

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from ray import tune

# Create a custom dataset class compatible with Hugging Face's Trainer
class Dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float).to(device)
        self.labels = torch.tensor(labels, dtype=torch.long).to(device)


    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        item = {'input_ids': self.features[idx], 'labels': self.labels[idx]}
        return item

train_dataset = Dataset(X_train, y_train)
test_dataset = Dataset(X_test, y_test)

# Function to set up the trainer for hyperparameter tuning
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained("NexaAIDev/Octopus-v2", num_labels=len(set(y)))
    return model
def hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-5, 1e-4),
        "num_train_epochs": tune.choice([3, 6, 9]),
        "per_device_train_batch_size": tune.choice([8, 16, 32])
    }
resources_per_trial = {"cpu": 2, "gpu": 0.5}  # Adjust based on your system's specification



c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester6\460j\460jFinalProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-24 00:00:28,261	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-24 00:00:28,517	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
# Hyperparameter space for tuning
def hp_space():
    return {
        "learning_rate": tune.loguniform(1e-5, 1e-4,1e-3),
        "num_train_epochs": tune.choice([3, 6, 9]),

    }

In [5]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    per_device_train_batch_size=2  # Starting point, tune will adjust
)


# Initialize Trainer with hyperparameter search
trainer = Trainer(
    args=training_args,
    model_init=model_init,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=None  # Define as needed
)

# Conduct hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=10,
    resources_per_trial=resources_per_trial
)

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester6\460j\460jFinalProject\.venv\Lib\site-packages\transformers\training_args.py:1454: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.96s/it]
Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at NexaAIDev/Octopus-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-04-24 00:00:57,804	INFO worker.py:

MemoryError: 